In [1]:
import os
import sys
sys.path.insert(0,"/home/kal/CTCF/modules/")
sys.path.insert(0,"/home/kal/CTCF/mass_CTCF/modules/")
os.environ['CUDA_VISIBLE_DEVICES'] = '1' # Must be before importing keras!
#import tf_memory_limit
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import precision_recall_curve
import ctcfgen
from convnet import Bias
import pickle
from keras.models import load_model, Model
from keras.utils import plot_model
import time
import keras.backend as K
import re

input_window = 256
batch_size = 32
min_pred = 0.5

model_path='/home/kal/CTCF/mass_CTCF/output/sigmoid/20170919_150754/20170919_150754_weights_3_248_0.91.hdf5'
gen_path='/home/kal/data/ctcfgen_data.hdf5'
history_path='/home/kal/CTCF/mass_CTCF/output/tanh/20170919_132022/20170919_132022_history'

Using TensorFlow backend.


InternalError: Failed to create session.

In [5]:
def score_loss(y_true, y_pred):
    preds = K.flatten(K.sigmoid(y_pred - 1))
    trues = K.flatten(y_true)
    return K.square(preds - trues)

In [11]:
# get a folder.
out_dir = os.path.dirname(model_path)

# Import the model params.
model = load_model(model_path, custom_objects={'Bias':Bias})
# Graph the model.
plot_model(model, to_file=os.path.join(out_dir, 'model_graph.png'), show_shapes=True)
# Get model training history.
with open(history_path + '1.pk1', 'rb') as input:
    h1 = pickle.load(input)
with open(history_path + '2.pk1', 'rb') as input:
    h2 = pickle.load(input)
with open(history_path + '3.pk1', 'rb') as input:
    h3 = pickle.load(input)    
    
# Get a generator.
gen = ctcfgen.CTCFGeneratorhdf5(gen_path)
print(str(gen.train_pos.shape[0]+gen.test_pos.shape[0]+gen.val_pos.shape[0]))


TypeError: arg 5 (closure) must be None or tuple

In [ ]:
def pairgen(mode='train'):
    p = gen.pos_training_gen(mode=mode)
    n = gen.neg_training_gen(mode=mode)
    labels = np.zeros(batch_size)
    labels[:(batch_size // 2)] = 1
    while True:
        pos_seqs = [ctcfgen.encode(next(p)) for i in range(batch_size // 2)]
        neg_seqs = [ctcfgen.encode(next(n)) for i in range(batch_size // 2)]
        yield np.asarray(pos_seqs + neg_seqs), labels[..., np.newaxis, np.newaxis]
g = pairgen(mode='test')

In [ ]:
# Get predicitons.
preds = model.predict_generator(g, gen.test_pos.shape[0] // batch_size).flatten()

# Get labels.
label_set = np.zeros(batch_size)
label_set[:(batch_size // 2)] = 1
num_batches = len(preds) / batch_size
labels = np.asarray(np.tile(np.asarray(label_set), (int(num_batches))))

In [ ]:
# Look at some predicted and actual values
ctcf = np.mean(np.ma.array(preds, mask=labels))
not_ctcf = np.mean(np.ma.array(preds, mask=np.invert(labels.astype(np.bool))))
plt.bar([0,1], [not_ctcf, ctcf])
plt.xlabel('CTCF Motif?')
plt.ylabel('Mean Predicted Value')
plt.ylim([0.0, 1.0])
plt.xticks([0,1], ['Yes', 'No'])
plt.show()

In [ ]:
# plot a pr curve
precision, recall, thresholds = precision_recall_curve(labels, preds, pos_label=1)
plt.clf()
plt.plot(recall, precision, label='Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.savefig(os.path.join(out_dir, 'prc.png'), bbox_inches='tight') 
plt.show()
print('done')

In [ ]:
# Plot a pr gain curve
prop_pos = sum(labels)/len(labels)
precision_gain = [(x-prop_pos)/((1-prop_pos)*x) for x in precision]
recall_gain = [(x-prop_pos)/((1-prop_pos)*x) for x in recall]

plt.clf()
plt.plot(recall_gain, precision_gain, label='Precision-Recall Gain Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.show()
plt.savefig(os.path.join(out_dir, 'prgc.png'), bbox_inches='tight')


In [ ]:
# Define procedure to write out fasta files.
def get_fasta(gen, path, label_op=lambda x: ('', 1), rejects=False):
    with open(path, 'w') as output:
        if rejects:
            wrong_output = open(path.rsplit('.', 1)[0] +' _rejects.' + path.rsplit('.', 1)[1], 'w')
        for seq in gen:
            label = label_op(seq)
            if label[1] >= min_pred and len(seq) > 0:
                output.write('>sequence|' + label[0] + '\n' + seq.tostring().decode('UTF-8') +'\n')
            elif rejects:
                wrong_output.write('>sequence|' + label[0] + '\n' + seq.tostring().decode('UTF-8') +'\n')
        if rejects:
            wrong_output.close()

In [ ]:
def group_stats(key, h1, h2, h3):
    # Summarize history for accuracy
    out1 = np.copy(h1[key])
    out2 = np.copy(h2[key])
    out3 = np.copy(h3[key])
    return np.concatenate([out1, out2, out3])

In [ ]:
# Summarize history for accuracy
plt.plot(group_stats('acc', h1, h2, h3))
plt.plot(group_stats('val_acc', h1, h2, h3))
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig(os.path.join(out_dir, 'model_accuracy.png'), bbox_inches='tight')

In [ ]:
# Summarize history for loss
plt.plot(group_stats('loss', h1, h2, h3))
plt.plot(group_stats('val_loss', h1, h2, h3))
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig(os.path.join(out_dir, 'model_loss.png'), bbox_inches='tight')

In [ ]:
val_acc = group_stats('val_acc', h1, h2, h3)
print(max(val_acc))
print(np.argmax(val_acc))

In [ ]:
# Make a graph of activations
get_per_base_output = K.function([model.layers[0].input, K.learning_phase()],
                                  [model.get_layer('conv1d_5').output])
p = gen.pos_training_gen(mode='test')
seq = next(p)
seq_letters = [char for char in seq.tostring()]
per_base_output = get_per_base_output([[ctcfgen.encode(seq) for i in range(batch_size)], 0])[0]
plt.plot(per_base_output[0])
plt.plot(per_base_output[33])
plt.title('per-base activations')
plt.ylabel('activation')
plt.xlabel('nucleotide')
plt.legend(['forward', 'reverse'], loc='upper left')
plt.show()
plt.savefig(os.path.join(out_dir, 'model_loss.png'), bbox_inches='tight')
actual_seq = seq.tostring().decode('UTF-8')
print(actual_seq)

In [ ]:
# Pull out peak-centered windows
p = gen.pos_training_gen(mode='test', once=True)
fcount, rcount = 0, 0
peak_seqs=[]
for seq in p:
    activations = get_per_base_output([np.tile(np.expand_dims(ctcfgen.encode(next(p)), axis=0), (batch_size,1,1)), 0])[0]
    fcenters = [idx for idx in range(len(activations[0])) if activations[0][idx]>1]
    rcenters = [idx for idx in range(len(activations[33])) if activations[33][idx]>1]
    for idx in fcenters:
        peak_seqs.append(seq[idx-20:idx+20])
    for idx in rcenters:
        peak_seqs.append(ctcfgen.rc(seq[idx-20:idx+20]))
    
    forward = activations[0]
    reverse = activations[33]
    if np.mean(forward) > np.mean(reverse):
        fcount += 1
    else:
        rcount += 1
        
# Write peak windows to a fasta.
get_fasta(peak_seqs, os.path.join(out_dir, 'activation_seqs.fa'))

# Plot predicted forward vs. backwards motifs
plt.bar([0,1], [fcount, rcount])
plt.xticks([0,1], ['forward', 'reverse'])
plt.show()

In [ ]:
# Check if any sequences have the motif from wikipedia 
# https://en.wikipedia.org/wiki/CTCF
g = gen.pos_training_gen(mode='test', once=True)
regex = re.compile('ccgcg.gg.ggcag')
for seq in g:
    if re.search(regex, seq.tostring().decode('UTF-8')):
        print(seq)